In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler, StandardScaler, MinMaxScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.model_selection import train_test_split
from tensorflow.keras import layers, losses
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.models import Model

# ECG
---
- An electrocardiogram (ECG) is a simple, non-invasive test that records the electrical activity of the heart.<br>
- An ECG can help diagnose certain heart conditions, including abnormal heart rhythms and coronary heart disease (heart attack and angina).<br>
- A doctor may recommend an ECG if you are experiencing symptoms like chest pain, breathlessness, dizziness, fainting or a feeling of your heart racing, fluttering, thumping or pounding in your chest (palpitations).<br>
- An ECG can also help monitor how treatments for a heart condition, like medicines or implantable cardiac devices, are working.<br>

# Diseases that are diagnosed with ECG
---
- Abnormal heart rhythms (arrhythmia).
- Heart inflammation (pericarditis or myocarditis).
- Ennlargement of the heart walls or viens (Cardiomyopathy).

# Step 1: Read the dataset
---
### About the Dataset
---
https://www.timeseriesclassification.com/description.php?Dataset=ECG5000

- The original dataset for "ECG5000" is a 20-hour long ECG downloaded from Physionet. The name is BIDMC Congestive Heart Failure Database(chfdb) and it is record "chf07". It was originally published in "Goldberger AL, Amaral LAN, Glass L, Hausdorff JM, Ivanov PCh, Mark RG, Mietus JE, Moody GB, Peng C-K, Stanley HE. PhysioBank, PhysioToolkit, and PhysioNet: Components of a New Research Resource for Complex Physiologic Signals. Circulation 101(23)". The data was pre-processed in two steps: (1) extract each heartbeat, (2) make each heartbeat equal length using interpolation. This dataset was originally used in paper "A general framework for never-ending learning from time series streams", DAMI 29(6). After that, 5,000 heartbeats were randomly selected. The patient has severe congestive heart failure and the class values were obtained by automated annotation


In [ ]:
dataframe = pd.read_csv('http://storage.googleapis.com/download.tensorflow.org/data/ecg.csv', header=None)
raw_data = dataframe.values
dataframe.head()

In [ ]:
dataframe.tail(10)

# Separate the target variable from the features.
---
- Normal rhythms, which are labeled in this dataset as 1. 
- Abnornmal heart rhythms are labeled as 0.

In [ ]:
# The last element contains the labels
labels = raw_data[:, -1]

In [ ]:
# Separate the normal rhythms from the abnormal rhythms.
# Train the autoencoder using only the normal rhythms.
# Test the autoencoder using only the abnormal rhythms
labels

# Separate the data to test and train
---

In [ ]:
# The other data points are the electrocadriogram data
data = raw_data[:, 0:-1]

train_data, test_data, train_labels, test_labels = train_test_split(
    data, labels, test_size=0.2, random_state=21
)

# Normalize the dataset
---
<font color=red><b>QUESTION</b></font>
- When to use normalization as oppose to standardization?
    - If there are different scales for the data and the machine learning algorithm used is sensitve to the scaling of the features, normalization is recommeneded to use over standardization.

In [ ]:
def transform(type, train, test):
    train_trans = None
    test_trans = None
    sc = None
    if type == "NOR":
        min_val = tf.reduce_min(train)
        max_val = tf.reduce_max(train)

        train = (train - min_val) / (max_val - min_val)
        test = (test - min_val) / (max_val - min_val)

        train_trans = tf.cast(train, tf.float32)
        test_trans = tf.cast(test, tf.float32)
    elif type == "STA":
        sc = StandardScaler()
        train = sc.fit_transform(train)
        test = sc.transform(test)

        train_trans = tf.cast(train, tf.float32)
        test_trans = tf.cast(test, tf.float32)
    elif type == "LOG":
        # Log transformation
        epsilon = 1e-8  # Small constant to avoid log(0)
        train = np.log1p(train + epsilon)
        test = np.log1p(test + epsilon)

        train_trans = tf.cast(train, tf.float32)
        test_trans = tf.cast(test, tf.float32)
    elif type == "MIN":
        sc = MinMaxScaler()
        train = sc.fit_transform(train)
        test = sc.transform(test)

        train_trans = tf.cast(train, tf.float32)
        test_trans = tf.cast(test, tf.float32)

    return train_trans, test_trans

In [ ]:
# Data normalization
train_data, test_data = transform('NOR', train_data, test_data)

In [ ]:
train_labels = train_labels.astype(bool)
test_labels = test_labels.astype(bool)

normal_train_data = train_data[train_labels]
normal_test_data = test_data[test_labels]

anomalous_train_data = train_data[~train_labels]
anomalous_test_data = test_data[~test_labels]

In [ ]:
plt.grid()
plt.plot(np.arange(140), normal_train_data[0])
plt.title("A Normal ECG")
plt.show()

In [ ]:
plt.grid()
plt.plot(np.arange(140), anomalous_train_data[0])
plt.title("An Anomalous ECG")
plt.show()

# Create the architecture of the autoencoder
---
- What are the parts and the purpose of each part of an autoencoder?
    - Encoder: Compresses input and produces the code
    - Code: Compressed version of the input
    - Decoder: Reconstructs the input from the compressed version

In [ ]:
class AnomalyDetector(Model):
  def __init__(self):
    super(AnomalyDetector, self).__init__()
    self.encoder = tf.keras.Sequential([
      layers.Dense(64, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(8, activation="relu"),
      layers.Dropout(0.001)])

    self.decoder = tf.keras.Sequential([
      layers.Dense(8, activation="relu"),
      layers.Dense(16, activation="relu"),
      layers.Dense(32, activation="relu"),
      layers.Dense(64, activation="relu"),
      layers.Dense(140, activation="sigmoid")])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

autoencoder = AnomalyDetector()

In [ ]:
autoencoder.compile(optimizer='adam', loss='mae')
# autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train your autoencoder architecture
---

In [ ]:
history = autoencoder.fit(normal_train_data, normal_train_data, 
          epochs=20, 
          batch_size=512,
          validation_data=(test_data, test_data),
          shuffle=True)

# Plot the errors
---

In [ ]:
plt.plot(history.history["loss"], label="Training Loss")
plt.plot(history.history["val_loss"], label="Validation Loss")
plt.legend()

# Question
---
- In reality trainnig loss should always be lower than validation loss, why?
    - It should always be lower than validation loss because it shows that the model was able to learn properly the characteristics and patterns of the data for it to perform well on the validation data or new and unseen data. But, it should not be too low since it would indicate overfitting.
- What happens if validation loss is lower than the training loss?
    - If validation loss is lower than the training loss, it could mean that the model was unable to learn properly the underlying patterns and characteristics of the testing data for it to adapt to new and unseen data, i.e., validation data.

# Performance Testing from Training Data
---
- We can use our trained autoencoders to reconstruct the ECG signals.
- Ideally out reconstructed signals shold look exactly the same as the training signal but in reality it is not due to some variation in the data.
- We then create a "reconstruction error" This is an area that determines how close or how far the reconstructed data is to the  datatraining.
- For medical settings the reconstruction error should be lower than 1 standard deviation of the training data.

#### Detecting anomalies
---
- Detect anomalies by calculating whether the reconstruction loss is greater than a fixed threshold. 
- Determine the mean average error for normal examples from the training set, then classify future examples as anomalous if the reconstruction error is higher than one standard deviation from the training set.
- Choose a threshold value that is one standard deviations above the mean.

In [ ]:
encoded_data = autoencoder.encoder(normal_test_data).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

plt.plot(normal_test_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], normal_test_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

In [ ]:
reconstructions = autoencoder.predict(normal_train_data)
train_loss = tf.keras.losses.mae(reconstructions, normal_train_data)

plt.hist(train_loss[None,:], bins=50)
plt.xlabel("Train loss")
plt.ylabel("No of examples")
plt.show()

In [ ]:
threshold = np.mean(train_loss) + np.std(train_loss)
print("Threshold: ", threshold)

# Generating Recommendations for Diagnosis (THIS IS <font color=red>NOT A DIAGNOSIS!</font>)
---
- Once the construction error is confirmed to be lower than 1 standard deviation form the training set, we can use the same concept towards the testing dataset.
- Plot the testing dataset, and reconstruct the signal from the decoded layer of the autoencoders, and generate a reconstruction error.

In [ ]:
encoded_data = autoencoder.encoder(anomalous_test_data).numpy()
decoded_data = autoencoder.decoder(encoded_data).numpy()

plt.plot(anomalous_test_data[0], 'b')
plt.plot(decoded_data[0], 'r')
plt.fill_between(np.arange(140), decoded_data[0], anomalous_test_data[0], color='lightcoral')
plt.legend(labels=["Input", "Reconstruction", "Error"])
plt.show()

If you examine the reconstruction error for the anomalous examples in the test set, you'll notice most have greater reconstruction error than the threshold. By varing the threshold, you can adjust the precision and recall of your classifier.

In [ ]:
reconstructions = autoencoder.predict(anomalous_test_data)
test_loss = tf.keras.losses.mae(reconstructions, anomalous_test_data)

plt.hist(test_loss[None, :], bins=50)
plt.xlabel("Test loss")
plt.ylabel("No of examples")
plt.show()

Classify an ECG as an anomaly if the reconstruction error is greater than the threshold.

In [ ]:
def predict(model, data, threshold):
  reconstructions = model(data)
  loss = tf.keras.losses.mae(reconstructions, data)
  return tf.math.less(loss, threshold)

def print_stats(predictions, labels):
  print("Accuracy = {}".format(accuracy_score(labels, predictions)))
  print("Precision = {}".format(precision_score(labels, predictions)))
  print("Recall = {}".format(recall_score(labels, predictions)))

In [ ]:
preds = predict(autoencoder, test_data, threshold)
print_stats(preds, test_labels)

# Coding Experiments
- Holding the autoencoder architecture constant. What would happen to the performance if you chosed other data transformation? Experiment using standardization, and log transformation, minmax scaler.
    - Normalization
        - Accuracy = 0.946
        - Precision = 0.994140625
        - Recall = 0.9089285714285714
    - Standardization
        - Accuracy = 0.643
        - Precision = 0.6236297198538368
        - Recall = 0.9142857142857143
    - Log transformation - error cause of NaN values
        - Accuracy = NaN
        - Precision = NaN
        - Recall = NaN
    - Minmax scaler 
        - Accuracy = 0.946
        - Precision = 0.9922178988326849
        - Recall = 0.9107142857142857
- Holding the autoencoder architecture constant. What would happen to the performance if the label data were not transformed into boolean?
    - The model would not be able to distinguish between normal and anomalous data during training if label data were not transformed into boolean
- Holding the autoencoder architecture constant. What would the behavior of the error reconstruction if a different optimizer was used? (choose only one optimizer from https://www.tensorflow.org/api_docs/python/tf/keras/optimizers)
    - If a different optimizer was used, the behavior of error reconstruction would be affected like its training speed and convergence rate.
- Holding the autoencoder architecture constant. Try using binary cross entropy as a loss function and compare its reconstruction error to the original loss function.
    - Mean Absolute Error
        - Accuracy = 0.946
        - Precision = 0.994140625
        - Recall = 0.9089285714285714
    - Binary Cross Entropy
        - Accuracy = 0.946
        - Precision = 0.9922178988326849
        - Recall = 0.9107142857142857
- Try experimenting with the autoencoder architecture and record the result, for which new architecture has the lowest reconstruction error? Try playing around with:
    - Activation function: Adam
    - Loss function: MAE
    - Dropout rate: 0.001
    - Units per layer:
        - Encoding = 64, 16, 16, 8
        - Decoding = 8, 16, 32, 64, 140
    - Number of layers:
        - Encoding = 5
        - Decoding = 5
    - Results:
        - Accuracy = 0.945
        - Precision = 0.9922027290448343
        - Recall = 0.9089285714285714